In [7]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import datetime
import PIL
import os
import argparse
import matplotlib.pyplot as plt
from CycleGAN import CycleGAN
from ImageDataset import ImageDataset
from torch.utils.data import DataLoader

### Hyperparameters

In [8]:
LR = 0.001
batch_size = 200
num_epochs = 10

### Helper Methods

In [9]:
def plot_graph(num_epochs,acc_list,loss_list):
    #usage : plot_graph(num_epochs,acc_list,loss_list)
    plt.ioff()
    fig = plt.figure()
    plt.subplot(2, 1, 1)
    plt.ylabel('Training loss')
    plt.plot(np.arange(num_epochs), loss_list, 'k-')
    plt.title('Training Loss and Training Accuracy')
    plt.xticks(np.arange(num_epochs, dtype=int))
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(np.arange(num_epochs), acc_list, 'b-')
    plt.ylabel('Training Accuracy')
    plt.xlabel('Epochs')
    plt.xticks(np.arange(num_epochs, dtype=int))
    plt.grid(True)
    plt.savefig("plot.png")
    plt.close(fig)

# Main

### Load DataSet

In [13]:
transform_train = transforms.Compose([transforms.RandomCrop(64, padding=4),transforms.RandomHorizontalFlip(p=2),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
transform_test = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
trainset = DataLoader(ImageDataset('../datasets/monet2photo/', transforms_=transform_train),
                        batch_size=batch_size, shuffle=True)
testset = DataLoader(ImageDataset('../datasets/monet2photo/', transforms_=transform_train,mode='test'),
                        batch_size=batch_size, shuffle=True)
print(len(trainset))
print(len(testset))

32
4


### Training

In [14]:
for epoch in range(1,num_epochs+1):
    pass